## Chapter 1 - Searching ElasticSearch

In [26]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/shakespeare/_search?pretty' -d '{"query" : {"match_phrase" : {"text_entry" : "to be or not to be"}}}'

{
  "took" : 67,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 1,
      "relation" : "eq"
    },
    "max_score" : 13.889601,
    "hits" : [
      {
        "_index" : "shakespeare",
        "_type" : "_doc",
        "_id" : "34229",
        "_score" : 13.889601,
        "_source" : {
          "type" : "line",
          "line_id" : 34230,
          "play_name" : "Hamlet",
          "speech_number" : 19,
          "line_number" : "3.1.64",
          "speaker" : "HAMLET",
          "text_entry" : "To be, or not to be: that is the question:"
        }
      }
    ]
  }
}


## Chapter 2 - Mapping and Indexing Data

### Mappings
    A mapping is a schema definition - elastisearch has reasonable defaults, but sometimes you need to customize them.

Example 

    !curl -XPUT 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies -d '{ "mappings" : { "properties" : { "year" : {"type": "date"}}}}'
***
### Common Mappings
***
#### Field types  
String, byte, short, integer, long, float, double, boolean, date

        "properties": {
            "user_id": {
                "type": 
        "long"
            }
        }

#### Field Index  
Do you want this field indexed for full-text search?  analyzed / not_analyzed / no

        "properties": {
            "genre": {
                "index": 
        "not_analyzed"
            }
        }

#### Field Analyzer
Define your tokenizer and token filter.  Standard / whitespace / simple / english etc.

        "properties": {
            "description": 
        {
                "analyzer": 
        "english"
            }
        }
***
### More About Analyzers
***
#### Character Filters
    Remove HTML encoding, convert & to and
    
#### Tokenizer
    Split strings on whitespace / punctuation / non-letters
    
#### Token Filter
    Lowercasing, stemming, synonyms, stopwords*
    
    *Stopwords might be difficult for searching phrases.

***
### Choices For Analyzers

#### Standard
    Splits on word boundaries, removes punctuation, lowercases.  Good choice if language is unknown.
#### Simple
    Splits on anything that isn't a letter, and lowercase.
#### Whitespace
    Splits on whitespace but doesn't lowercase.
#### Language
    Accounts for language-specific stopwords and stemming.
***

### Importing a Single Movie (JSON/REST)

In [28]:
!curl -H "Content-Type: application/json" -XPUT 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487' -d '{"genre":["IMAX","Sci-Fi"], "title": "Interstellar", "year": 2014}'

{"_index":"movies","_type":"_doc","_id":"109487","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}

In [29]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_mapping' 

{"movies":{"mappings":{"properties":{"genre":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"title":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"year":{"type":"long"}}}}}

In [30]:
!curl -H "Content-Type: application/json" -XPOST 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487' -d '{"genre":["IMAX","Sci-Fi"], "title": "Interstellar", "year": 2014}'

{"_index":"movies","_type":"_doc","_id":"109487","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}

In [31]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_search?pretty' 

{
  "took" : 944,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 1,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "movies",
        "_type" : "_doc",
        "_id" : "109487",
        "_score" : 1.0,
        "_source" : {
          "genre" : [
            "IMAX",
            "Sci-Fi"
          ],
          "title" : "Interstellar",
          "year" : 2014
        }
      }
    ]
  }
}


### Import Many Documents using Bulk

In [84]:
#!curl -H "Content-Type: application/json" -XPUT 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/_bulk?pretty' --data-binary @movies.json
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/_search?pretty'

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 10000,
      "relation" : "gte"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "movies",
        "_type" : "_doc",
        "_id" : "109487",
        "_score" : 1.0,
        "_source" : {
          "genre" : [
            "IMAX",
            "Sci-Fi"
          ],
          "title" : "Interstellar",
          "year" : 2014
        }
      },
      {
        "_index" : "movies",
        "_type" : "_doc",
        "_id" : "135569",
        "_score" : 1.0,
        "_source" : {
          "id" : "135569",
          "title" : "Star Trek Beyond",
          "year" : 2016,
          "genre" : [
            "Action",
            "Adventure",
            "Sci-Fi"
          ]
        }
      },
      {
        "_index" : "movies",
        "_type" : "_doc",
        "_id" : "122886",
        "_score"

***
### Updating Data in Elasticsearch 
#### Versions
  * Every document has a _version field 
  * Elasticsearch documents are __immutable__ 
  * When you update an existing document: 
      * a new document is created with an incremented _version 
      * the old document is marked for deletion 
      
#### Partial Update API
    curl -H "Content-Type: application/json" -XPOST 127.0.0.1:9200/movies/_doc/109487/_update -d ' 
        { 
            "doc":{ 
                "title": "Interstellar" 
            } 
            }'

In [85]:
!curl -H "Content-Type: application/json" -XPUT  'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487?pretty' -d '{"genres": ["IMAX", "Sci-Fi"], "title": "Interstellar foo","year":2014}'

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "109487",
  "_version" : 3,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 6,
  "_primary_term" : 1
}


In [86]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487?pretty' 

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "109487",
  "_version" : 3,
  "_seq_no" : 6,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "genres" : [
      "IMAX",
      "Sci-Fi"
    ],
    "title" : "Interstellar foo",
    "year" : 2014
  }
}


In [88]:
!curl -H "Content-Type: application/json" -XPOST  'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487/_update' -d '{"doc": {"title": "Interstellar","year":2014}}'

{"_index":"movies","_type":"_doc","_id":"109487","_version":4,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":7,"_primary_term":1}

In [89]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/109487?pretty' 

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "109487",
  "_version" : 4,
  "_seq_no" : 7,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "genres" : [
      "IMAX",
      "Sci-Fi"
    ],
    "title" : "Interstellar",
    "year" : 2014
  }
}


***
### Deleting Data in Elasticsearch
    Just use the DELETE method:
        curl -XDELETE 127.0.0.1:9200/movies/_doc/58559
        
 

In [90]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_search?q=Dark' 

{"took":704,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":1.5169399,"hits":[{"_index":"movies","_type":"_doc","_id":"58559","_score":1.5169399,"_source":{ "id": "58559", "title" : "Dark Knight, The", "year":2008 , "genre":["Action", "Crime", "Drama", "IMAX"] }}]}}

In [91]:
!curl -H "Content-Type: application/json" -XDELETE 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/58559?pretty' 

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "58559",
  "_version" : 2,
  "result" : "deleted",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 8,
  "_primary_term" : 1
}


In [92]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_search?q=Dark'

{"took":4,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":0,"relation":"eq"},"max_score":null,"hits":[]}}

***
### Summary of Insert/Update/Delete

In [93]:
!curl -H "Content-Type: application/json" -XPUT 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/200000?pretty' -d '{"title": "Jons movie in Elastic", "genres":["Documentary"], "year": 2019}'

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "200000",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 9,
  "_primary_term" : 1
}


In [94]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/200000?pretty'

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "200000",
  "_version" : 1,
  "_seq_no" : 9,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "title" : "Jons movie in Elastic",
    "genres" : [
      "Documentary"
    ],
    "year" : 2019
  }
}


In [95]:
!curl -H "Content-Type: application/json" -XPOST 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/200000/_update' -d '{"doc": {"genres":["Documentary", "Comedy"]}}'

{"_index":"movies","_type":"_doc","_id":"200000","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":10,"_primary_term":1}

In [96]:
!curl -H "Content-Type: application/json" -XGET 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/200000?pretty'

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "200000",
  "_version" : 2,
  "_seq_no" : 10,
  "_primary_term" : 1,
  "found" : true,
  "_source" : {
    "title" : "Jons movie in Elastic",
    "genres" : [
      "Documentary",
      "Comedy"
    ],
    "year" : 2019
  }
}


In [97]:
!curl -H "Content-Type: application/json" -XDELETE 'ec2-13-59-84-109.us-east-2.compute.amazonaws.com:9200/movies/_doc/200000?pretty'

{
  "_index" : "movies",
  "_type" : "_doc",
  "_id" : "200000",
  "_version" : 3,
  "result" : "deleted",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "_seq_no" : 11,
  "_primary_term" : 1
}


### Dealing with Concurrency